预处理
===

在你对数据集进行模型训练之前，需要将其预处理成预期的模型输入格式。无论你的数据是文本、图像还是音频，它们都需要被转换并组装成一批批的张量。Transformers提供了一组预处理类，帮助你为模型准备数据。在本教程中，你将了解到：

* 文本，使用标记器(Tokenizer)将文本转换为一连串的标记，创建标记的数字表示，并将它们组装成张量。
* 语音和音频，使用特征提取器(Feature extractor)从音频波形中提取顺序特征，并将其转换为张量。
* 图像输入使用一个ImageProcessor将图像转换成张量。
* 多模态输入，使用一个处理器来结合标记器和特征提取器或图像处理器。

> AutoProcessor总能发挥作用，为你使用的模型自动选择正确的类，无论你使用的是标记器、图像处理器、特征提取器还是处理器。

在你开始之前，请安装Datasets，这样你就可以加载一些数据集进行实验：

In [ ]:
pip install datasets

## 自然语言处理(Natural Language Processing)

对文本数据进行预处理的主要工具是标记器。标记器根据一组规则将文本分割成标记。这些标记被转换成数字，然后被转换成张量，成为模型的输入。模型所需的任何额外输入都由标记器添加。

> 如果你打算使用一个预训练的模型，那么使用相关的预训练的标记器是很重要的。这确保了文本的分割方式与预训练语料库相同，并在预训练过程中使用相同的对应标记-索引（通常被称为词汇）。

通过AutoTokenizer.from_pretrained()方法加载一个预训练的标记符。这将下载一个模型预训练过的词汇：

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

然后将你的文本传递给标记器：

In [ ]:
encoded_input = tokenizer("Do not meddle in the affairs of wizards, for they are subtle and quick to anger.")
print(encoded_input)

```
{'input_ids': [101, 2079, 2025, 19960, 10362, 1999, 1996, 3821, 1997, 16657, 1010, 2005, 2027, 2024, 11259, 1998, 4248, 2000, 4963, 1012, 102], 
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
```

tokenizer返回一个有三个重要项目的字典：

* input_ids是对应于句子中每个标记的索引。
* attention_mask表示一个标记是否应该被关注。
* token_type_ids在有多个序列的情况下确定一个标记属于哪个序列。

通过对input_ids的解码，返回你的输入：

In [ ]:
tokenizer.decode(encoded_input["input_ids"])

```
'[CLS] Do not meddle in the affairs of wizards, for they are subtle and quick to anger. [SEP]'
```

正如你所看到的，标记化器在该句中添加了两个特殊的标记-CLS和SEP（分类器和分离器）。并非所有的模型都需要特殊的标记，但如果它们需要，标记器会自动为你添加它们。

如果有几个你想预处理的句子，把它们作为一个列表传给标记器：

In [ ]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_inputs = tokenizer(batch_sentences)
print(encoded_inputs)

```
{'input_ids': [[101, 1252, 1184, 1164, 1248, 6462, 136, 102], 
               [101, 1790, 112, 189, 1341, 1119, 3520, 1164, 1248, 6462, 117, 21902, 1643, 119, 102], 
               [101, 1327, 1164, 5450, 23434, 136, 102]], 
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0], 
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                    [0, 0, 0, 0, 0, 0, 0]], 
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], 
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 
                    [1, 1, 1, 1, 1, 1, 1]]}
```

### 填充（Pad）

句子并不总是相同的长度，这可能是一个问题，因为张量，即模型输入，需要有一个统一的形状。填充是一种策略，通过向较短的句子添加特殊的填充标记来确保张量是矩形的。

将padding参数设置为True，以填充批次中较短的序列，以匹配最长的序列：

In [ ]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True)
print(encoded_input)

```
{'input_ids': [[101, 1252, 1184, 1164, 1248, 6462, 136, 102, 0, 0, 0, 0, 0, 0, 0], 
               [101, 1790, 112, 189, 1341, 1119, 3520, 1164, 1248, 6462, 117, 21902, 1643, 119, 102], 
               [101, 1327, 1164, 5450, 23434, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0]], 
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 
                    [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]}
```

第一句和第三句现在用0来填充，因为它们比较短。

### 截取（Truncation）

另外一种情况，有时一个序列可能太长，模型无法处理。在这种情况下，你需要将序列截断成一个较短的长度。

将参数truncation设置为True，以将序列截断到模型所能接受的最大长度：

In [ ]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True)
print(encoded_input)

```
{'input_ids': [[101, 1252, 1184, 1164, 1248, 6462, 136, 102, 0, 0, 0, 0, 0, 0, 0], 
               [101, 1790, 112, 189, 1341, 1119, 3520, 1164, 1248, 6462, 117, 21902, 1643, 119, 102], 
               [101, 1327, 1164, 5450, 23434, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0]], 
 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 
                    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 
 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], 
                    [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 
                    [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]}
```

> 查看[Padding and truncation](https://huggingface.co/docs/transformers/pad_truncation)的概念指南，了解更多不同的填充和截断参数。

### 构建张量(Build tensors)

最后，你想让标记器返回实际的张量，这些张量被送入模型。

将return_tensors参数设置为PyTorch的pt，或TensorFlow的tf：

In [ ]:
batch_sentences = [
    "But what about second breakfast?",
    "Don't think he knows about second breakfast, Pip.",
    "What about elevensies?",
]
encoded_input = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt")
print(encoded_input)

```
{'input_ids': tensor([[101, 1252, 1184, 1164, 1248, 6462, 136, 102, 0, 0, 0, 0, 0, 0, 0],
                      [101, 1790, 112, 189, 1341, 1119, 3520, 1164, 1248, 6462, 117, 21902, 1643, 119, 102],
                      [101, 1327, 1164, 5450, 23434, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0]]), 
 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                           [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 
 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                           [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
                           [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]])}
```

## 音频

对于音频任务，你需要一个特征提取器来为模型准备你的数据集。特征提取器被设计用来从原始音频数据中提取特征，并将其转换为张量。

加载MInDS-14数据集（参见数据集教程，了解如何加载数据集的更多细节），看看你如何用音频数据集使用特征提取器：

In [ ]:
from datasets import load_dataset, Audio

dataset = load_dataset("PolyAI/minds14", name="en-US", split="train")

访问音频列的第一个元素，看一下输入的情况。 调用音频列会自动加载并重新采样音频文件：

In [ ]:
dataset[0]["audio"]

```
{'array': array([ 0.        ,  0.00024414, -0.00024414, ..., -0.00024414,
         0.        ,  0.        ], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~JOINT_ACCOUNT/602ba55abb1e6d0fbce92065.wav',
 'sampling_rate': 8000}
```

这将返回三个项目：
* array 是作为一个一维阵列加载的语音信号--并有可能重新采样。
* path 指向音频文件的位置。
* sampling_rate 是指每秒钟测量多少个语音信号的数据点。

在本教程中，你将使用Wav2Vec2模型。看一下模型卡，你会知道Wav2Vec2是在16kHz采样的语音音频上进行预训练的。重要的是，你的音频数据的采样率与用于预训练模型的数据集的采样率相匹配。如果你的数据的采样率不一样，那么你需要重新采样你的数据。

1. 使用Datasets的cast_column方法将采样率提高到16kHz：

In [ ]:
dataset = dataset.cast_column("audio", Audio(sampling_rate=16_000))

2. 再次调用音频列以重新取样音频文件：

In [ ]:
dataset[0]["audio"]

```
{'array': array([ 2.3443763e-05,  2.1729663e-04,  2.2145823e-04, ...,
         3.8356509e-05, -7.3497440e-06, -2.1754686e-05], dtype=float32),
 'path': '/root/.cache/huggingface/datasets/downloads/extracted/f14948e0e84be638dd7943ac36518a4cf3324e8b7aa331c5ab11541518e9368c/en-US~JOINT_ACCOUNT/602ba55abb1e6d0fbce92065.wav',
 'sampling_rate': 16000}
```

接下来，加载一个特征提取器，对输入进行规范化处理和填充。当对文本数据进行填充时，对于较短的序列会添加一个0。这个想法同样适用于音频数据。特征提取器默认在数组中添加0。

用AutoFeatureExtractor.from_pretrained()加载特征提取器：

In [ ]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base")

将音频阵列传递给特征提取器。我们还建议在特征提取器中加入sampling_rate参数，以便更好地调试，避免可能出现的任何潜在错误。

In [ ]:
audio_input = [dataset[0]["audio"]["array"]]
feature_extractor(audio_input, sampling_rate=16000)

```
{'input_values': [array([ 3.8106556e-04,  2.7506407e-03,  2.8015103e-03, ...,
        5.6335266e-04,  4.6588284e-06, -1.7142107e-04], dtype=float32)]}
```

就像标记器一样，你可以应用填充或截断来处理一个批次中的可变序列。看一下这两个音频样本的序列长度：

In [ ]:
dataset[0]["audio"]["array"].shape  # (173398,)

dataset[1]["audio"]["array"].shape  # (106496,)

创建一个函数来预处理数据集，使音频样本具有相同的长度。指定一个最大的样本长度，特征提取器将填充或截断序列以匹配它：

In [ ]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=16000,
        padding=True,
        max_length=100000,
        truncation=True,
    )
    return inputs

对数据集中的前几个例子应用preprocess_function：

In [ ]:
processed_dataset = preprocess_function(dataset[:5])

现在，样本的长度是一样的，与指定的最大长度相匹配。你现在可以把你处理过的数据集传递给模型了!

In [ ]:
processed_dataset["input_values"][0].shape  # (100000,)

processed_dataset["input_values"][1].shape  # (100000,)